In [1]:
from baseline import transform_test_data
from eval import evaluate_clusters_iou, count_matches, evaluate_clusters_entropy
from api_utils import run_inference
from prompt_lib import agent_prompt, meta_prompt, reflect_prompt, reflect_prompt_2
from meta_utils import run_meta_inference, run_agent_inference
import json
import numpy as np

In [2]:
%env TOGETHER_API_KEY=eb25e0d054aec138257edf3c264089a5fa04be265ce8c3890597884cec1bc622

env: TOGETHER_API_KEY=eb25e0d054aec138257edf3c264089a5fa04be265ce8c3890597884cec1bc622


In [3]:
def average_iou(predicted_answers, test_data):
    ious = []
    for i in range(len(test_data)):
        pred_clusters = predicted_answers[i]
        true_clusters = test_data[i]['solutions']
        print(pred_clusters)
        print(true_clusters)
        iou = evaluate_clusters_iou(pred_clusters=pred_clusters, true_clusters=true_clusters)
        ious.append(iou)
    avg_iou = sum(ious) / len(ious)
    return avg_iou

In [4]:
with open('data/splits/connections_test.json', 'r') as file:
    data = json.load(file)
test_data = transform_test_data(data, False)

predicted_answers = await run_inference(test_data, agent_prompt, model="databricks/dbrx-instruct")

with open('data/meta/iter_1_answers.json', 'w') as file:
    json.dump(predicted_answers, file)

print("average IOU loss in test set for iteration 1: ", average_iou(predicted_answers, test_data))


for i in range(2,8):
    with open(f'data/meta/iter_{i-1}_answers.json', 'r') as file:
        agent_responses = json.load(file)

    meta_instructions = await run_meta_inference(prompt = meta_prompt, instructions = agent_prompt, games = test_data, responses = agent_responses, model="databricks/dbrx-instruct")

    #predicted_answers = await run_agent_inference(prompt = reflect_prompt, instructions = agent_prompt, games = test_data, responses = agent_responses, meta_instructions = meta_instructions, model="databricks/dbrx-instruct")
    predicted_answers = await run_agent_inference(prompt = reflect_prompt_2, instructions = agent_prompt, games = test_data, responses = agent_responses, meta_instructions = meta_instructions, model="databricks/dbrx-instruct")

    with open(f'data/meta/iter_{i}_answers.json', 'w') as file:
        json.dump(predicted_answers, file)
    print("average IOU loss in test set for iteration ", i, ": ", average_iou(predicted_answers, test_data))

[[['LOOK', 'SIGHT', 'BLUE', 'DRESS'], ['LIKEWISE', 'SAME', 'MANNER', 'STYLE'], ['SAILOR', 'TOUCH', 'SMELL', 'HARVEST'], ['NEW', 'SECOND', 'DITTO', 'TASTE']], [['SNOW', 'HAIL', 'DEEP', 'EXTREME'], ['FIRE', 'STATIC', 'NOISE', 'BLOODY'], ['RAM', 'BUMP', 'KNOCK', 'FIERCE'], ['VIRGIN', 'PROUD', 'BUTT', 'FUZZ'], [''], ['Explanation'], [''], ['SNOW', 'HAIL', 'DEEP', 'EXTREMEThesewordsarerelatedtoweatherconditions'], ['FIRE', 'STATIC', 'NOISE', 'BLOODYThesewordscanformcommonphraseswhenpairedwithanotherwordeg', 'FireNOISE', 'StaticFIRE', 'BloodyNOISE'], ['RAM', 'BUMP', 'KNOCK', 'FIERCEThesewordsareassociatedwithacertainverbphraseoractioneg', 'RamBUMP', 'KnockFIERCE'], ['VIRGIN', 'PROUD', 'BUTT', 'FUZZThesewordsarerelatedtobodypartsorpersonalattributeseg', 'VirginBUTT', 'ProudFUZZ']], [['MIST', 'FOG', 'HAZE', 'CLOUD'], ['FIN', 'TAIL', 'FLIPPER', 'BALL'], ['SHADOW', 'TRAIL', 'TRACK', 'ICE'], ['PLUNGER', 'NETHER', 'IRE', 'BUMPER']]]
[['LOOK', 'SIGHT', 'BLUE', 'DRESS'], ['LIKEWISE', 'SAME', 'MANNER